# Introduction

Here my goal is to introduce a basic stop loss and take profit logic using pandas (so i can leverage cuDF for backtests) that I can apply to any strategy.

In [1]:
import pandas as pd
import numpy as np

# generate sample data
np.random.seed(42)
data = np.random.randn(100).cumsum() + 100
df = pd.DataFrame({'close': data})

# compute SMA with a window of 20
window = 20
df['SMA'] = df['close'].rolling(window=window).mean()

# parameters for stop loss (5%) and take profit (10%)
stop_loss_pct = 0.05
take_profit_pct = 0.10

# vectorized entry: flag a new trade only when price crosses above SMA
df['new_entry'] = (df['close'] > df['SMA']) & (df['close'].shift(1) <= df['SMA'].shift(1))

# Set entry price only at new entry points, then forward-fill
df['entry_price'] = np.where(df['new_entry'], df['close'], np.nan)
df['entry_price'] = df['entry_price'].ffill()

# Compute exit condition based on the entry price:
# exit if price <= entry*(1 - stop_loss) OR price >= entry*(1 + take_profit) OR price < SMA
exit_sl = df['close'] <= df['entry_price'] * (1 - stop_loss_pct)
exit_tp = df['close'] >= df['entry_price'] * (1 + take_profit_pct)
exit_sma = df['close'] < df['SMA']
df['exit_condition'] = exit_sl | exit_tp | exit_sma

# Create a signal column based on vectorized conditions:
# Issue 'Buy' when a new entry is flagged; issue 'Sell' when the exit condition is met.
df['signal'] = np.where(df['new_entry'], 'Buy', '')
df.loc[df['exit_condition'], 'signal'] = 'Sell'

# Generate a basic position flag: 1 at buy, -1 at exit, 0 elsewhere
df['position'] = np.where(df['signal'] == 'Buy', 1, np.where(df['signal'] == 'Sell', -1, 0))

# Optional: clear the entry price after selling to start fresh
df.loc[df['signal'] == 'Sell', 'entry_price'] = np.nan

print(df.tail(10))

        close        SMA  new_entry  entry_price  exit_condition signal  \
90  91.204340  91.350846      False          NaN            True   Sell   
91  92.172985  91.336016       True    92.172985           False    Buy   
92  91.470932  91.287875      False    92.172985           False          
93  91.143269  91.145119      False          NaN            True   Sell   
94  90.751161  91.113744      False          NaN            True   Sell   
95  89.287646  90.968099      False          NaN            True   Sell   
96  89.583767  90.832907      False          NaN            True   Sell   
97  89.844822  90.725718      False          NaN            True   Sell   
98  89.849935  90.614197      False          NaN            True   Sell   
99  89.615348  90.590325      False          NaN            True   Sell   

    position  
90        -1  
91         1  
92         0  
93        -1  
94        -1  
95        -1  
96        -1  
97        -1  
98        -1  
99        -1  


# Detailed Explanation of the Vectorized SMA Strategy

1. Imported pandas and numpy and generated cumulative sample price data.
2. Calculated a 20-period Simple Moving Average (SMA) for the close prices.
3. Defined stop loss (5%) and take profit (10%) thresholds.
4. Detected new entry signals when the close price crosses above the SMA compared to the previous row, marking these events as 'Buy'.
5. Captured the entry price at the moment of a buy signal and forward-filled it for subsequent rows.
6. Computed exit conditions in a vectorized manner: exit is signaled if the price falls to or below the stop loss, rises to or above take profit, or falls below the SMA. This triggers a 'Sell' signal.
7. Assigned position flags (1 for buy, -1 for sell, 0 otherwise) based on these signals, and reset the entry price upon selling.

This entirely vectorized approach eliminates the need for for-loops, leveraging pandas and numpy for efficient computation.